# Intro to Dolo: Adding a catastrophic shock to the RBC

## Computational Workshop at the Chilean Central Bank 2022

Pablo Winant

---

# Quick presentation of Dolo

## What is it ?

- Dynare for nonlinear models but:
    - much more modular
    - several methods to solve models
    - based on a complete opensource stack    
- Opensource:
    - developped during my thesis
    - then at the IMF
    - then at the BOE
- Main contributors:
    - Spencer Lyon (NYU)
    - Anastasia Zhutova (BdF)
    - James Graham (NYU)
    - Ezgi Ozsogut (PSE)
    - Norman Rion (Cergy)
    - Gabrielle Queran (Polytechnique/INSEE)
    - anybody is welcome to help / report bugs / ask questions

Goals: 

- separate model definitions from the solution routines
- also leverage the programming language for anything that is not the model:
    - graphs
    - loops, ...
- aim for high performance generic algorithms
    - JIT, GPU, ...
- model description is rather generic
    - currently: any kind of shock (iid, autocorrelated, discrete)
    - future: any kind of shock, any kind of control

## The YAML file

- uses YAML standard
    - extended with a mini-language to create objects
- contains:
    - declarations
    - equations
        - timing similar to dynare but with t and square brackets
        - they have a formal grammar syntax
    - calibration section
    - exogenous shock
    - calibration
    - options

## Examples

Examples can be found in dolo_models (WIP) and in the examples/models subdirectory.
Check documentation online.

## Test everything is working

- Check the content of `models/rbc.yaml`



- Import it with `yaml_import`.  (make sure you are in the right directory)


In [ ]:
model = yaml_import("models/rbc.yaml")


- Use the model object to compute steady-state residuals. (Check `model.calibration` and )



- Compute linear perturbation solution `perturb`



- Simulate the solution with `simulate` (note that the resulting object is an axisarray)



- Compute nonlinear solution `time_iteration`



- Compare decision rules (using `tabulate`)

# Add a catastrophic shock to the RBC model

The goal here is to add a discrete, catastrophic shock to the RBC model.

In the orginal implementation, the law of motion for the capital is:

  $$k_t = (1-\delta)k_{t-1} + i_{t-1}$$

The productivity process depends on an AR1:

  $$z_t = \rho z_{t-1} + \epsilon_{z,t}$$

and is given by:

  $$A_t = \exp(z_t)$$
  
We want to add two additional Markov states $x_t$ and $y_t$ defined as follows:

- $x_t$ is equal to 1 in the first period the disaster hits the economy
- $y_t$ is equal to 1 as long as the economy has not recovered (probability $p$ of staying in the disaster state)
  
When there is no disaster ($x_t=y_t=0$) there is a probability $u=3\%$ that a disaster happens next period.

Given the parameters $c_k=30\%$ and  $c_z = 1 - \left( 1- c_k \right)^{1-\alpha}$ the equations are amended as follows:

$$k_t = (1-\delta)k_{t-1}*(1-c_k x_t)+ i_{t-1}$$

$$A_t = \exp(z_t)*(1-c_z y_t)$$

Save `rbc.yaml` as `rbc_disaster.yaml` and amend the model to allow for a disaster shock. The following parts need to be changed:
- the `symbols` definitions
- the equations
- the calibration (to add the values of `x` and `y`)


Import, solve and simulate the new version.

Use `time_iteration`, `tabulate`, `simulate`
